In [7]:
import pandas as pd
import numpy as np
import pathlib 
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
import warnings
import pandas as pd
from sklearn.metrics import roc_curve, auc
import pickle
def check_read_inside_region(start, seq, region):
        read_end = start + len(seq)
        region_start = int(region.split(":")[1].split("-")[0])
        region_end = int(region.split(":")[1].split("-")[1])
        if start >= region_start and read_end <= region_end:
            return "in"
        else: 
            return "overlap"

warnings.filterwarnings('ignore')

data_version = "TMD_cov"
output_version = "20240910"

outdir = "/media/hieunguyen/GSHD_HN01/outdir"
PROJECT = "TMD450_TCGA_data_analysis"
thres_hypo = 0.3
thres_hyper = 0.6

mode = "all"
input_cancer_class = "PAN_CANCER"
selected_candi_features = "ratio_raw"

path_to_main_output = os.path.join(outdir, PROJECT, output_version)
path_to_02_output = os.path.join(path_to_main_output, "PANCANCER02_output")
    
path_to_07_output = os.path.join(path_to_main_output, "PANCANCER07_output")
path_to_08_output = os.path.join(path_to_main_output, "PANCANCER08_output")

path_to_tf_output = os.path.join(outdir, PROJECT, output_version, "TF_output")
path_to_03_tf_output = os.path.join(path_to_tf_output, "03_output")
os.system(f"mkdir -p {path_to_03_tf_output}")

candidf = pd.read_excel(os.path.join(path_to_08_output, "candi_reads_all_{}_vs_control.xlsx".format("PAN_CANCER")))
candidf_origin = candidf.copy()
convert_ratio = {
    '0001': 0.001, 
    '001': 0.01, 
    '0005': 0.005, 
    '005': 0.05, 
    '01': 0.1,
    '05': 0.5,
    '08': 0.8
}

metadata_spike_in = pd.read_csv("metadata_Spike_in_silico_TMD_samples_from_tissue_highdepth_07102024.csv")


print(f"Number of samples in this version: {candidf.shape[0]}")
ichorcnadf = pd.read_csv("validation_Truong_Vi/metadata_Spike_in_silico_GW_samples_from_tissue_highdepth_15112024.csv")
ichorcnadf_bosung = pd.read_csv("summary_tumor_fraction_bosung_80pct.csv")
new_metadata = pd.read_csv("metadata_Spike_in_silico_GW_samples_from_tissue_highdepth_15112024.csv")
new_metadata  = new_metadata[new_metadata["Spike_in_ratio"] == 80]
new_metadata["ichorCNA"] = new_metadata["SampleID"].apply(lambda x: ichorcnadf_bosung[ichorcnadf_bosung["Sample"] == x]["ichorCNA"].values[0])
ichorcnadf = pd.concat([ichorcnadf, new_metadata], axis = 0)

candidf = candidf.merge(ichorcnadf[["SampleID", "ichorCNA"]], right_on = "SampleID", left_on = "SampleID")
candidf["Label"] = candidf["SampleID"].apply(lambda x: x.split("_")[8])
candidf["spike_in_ratio"] = candidf["SampleID"].apply(lambda x: convert_ratio[x.split("_")[4]])
print(f"Number of samples in this version: {candidf.shape[0]}")

metadata = pd.concat([metadata_spike_in, new_metadata], axis = 0)


Number of samples in this version: 4956
Number of samples in this version: 4056


In [12]:
candidf.groupby("spike_in_ratio")["SampleID"].count()

spike_in_ratio
0.001    750
0.005    750
0.010    750
0.050    750
0.100    732
0.500    234
0.800     90
Name: SampleID, dtype: int64